# Lab 4. Probabilistic Inference
# Task 4.3 Monty Hall Problem Using Bayesian Network
## Problem Descriptions
In the Monty Hall problem, there are 2 doors with goats behind and 1 door with a car behind. The contestant needs to choose the door with the car behind to win the prize. First, the contestant chooses a door out of 3 doors, and the host who knows what is behind the doors, opens another door that has a goat. Now there are only 2 doors left and the contestant needs to decide to stick to original choice, or switch to another door.

In this task, we calculate the probability of a prize behind each door using a Bayesian network. The Bayesian network for the Monty Hall problem can be defined as follows.

---
1. Definition of variables
 * Contestant's choice (C): Door chosen by the contestant at the beginning. This variable has 3 possible values {Door 0, Door 1, Door 2}.

 * Door with the prize (P): The door with the prize behind it, this variable also has 3 possible values {Door 0, Door 1, Door 2}.

 * Host's Choice (H): The door the host opens. The host will always open a door that does not have the prize and was not picked by the contestant. This variable also has three possible values {Door 0, Door 1, Door 2}.

2. Construction of the Bayesian network
 * P → H: The host's choice of which door to open depends on where the prize is. He must open the door that has a goat behind.
  
 * C → H: The host's choice also depends on the contestant's initial choice. He must open the door that was not picked by the contestant.

3. Prior Probabilities
    * P(P): The probability distribution of the prize being each door is the same, so it is 1/3 possibility of prize for each door.
   
   
      P(Prize = Door 0) = P(Prize = Door 1) = P(Prize = Door 2) = 1/3

 * P(C): The contestant's initial choice is also equally likely to be any of the three doors.


      P(Choice = Door 0) = P(Choice = Door 1) = P(Choice = Door 2) = 1/3

4. Conditional Probabilities

P(H | P, C): Given the door with the prize behind and the contestant's initial choice, the probability of the host opening the door with a goat varies in 2 possible situations. Let's assume the door with the prize behind is door 1.
 * Situation 1: The contestant chooses the prize door (P=C) at the beginning, and the host can open either of the remaining 2 doors with the goat behind. Since there are only 2 remaining doors, the probability would be 50%.

        P(H = Door 0 | P = Door 1, C = Door 1) = 1/2
        P(H = Door 1 | P = Door 1, C = Door 1) = 0
        P(H = Door 2 | P = Door 1, C = Door 1) = 1/2


* Situation 2: The contestant doesn't choose the prize door (P≠C), the host has only one choice (the door that is neither P nor C), so the probability would be 100%. If the contestant opens door 0, the host can only open door 2; If the contestant opens door 2, the host can only choose door 0.
        If the contestant chooses door 0,
        P(H = Door 0 | P = Door 1, C = Door 0) = 0
        P(H = Door 1 | P = Door 1, C = Door 0) = 0
        P(H = Door 2 | P = Door 1, C = Door 0) = 1

        If the contestant chooses door 2,
        P(H = Door 0 | P = Door 1, C = Door 2) = 1
        P(H = Door 1 | P = Door 1, C = Door 2) = 0
        P(H = Door 2 | P = Door 1, C = Door 2) = 0


## Implementation and Results

In [ ]:
!pip install pgmpy
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.2 MB/s eta 0:00:00


In [ ]:
# Defining the network structure
model = BayesianModel([('Contestant', 'Host'), ('Prize', 'Host')])

# Defining the CPDs:
#cpd_c = TabularCPD('Contestant', 3, [[1/3], [1/3], [1/3]])
#cpd_p = TabularCPD('Prize', 3, [[1/3], [1/3], [1/3]])
#cpd_h = TabularCPD('Host', 3, [[0, 0, 0, 0, 0.5, 1, 0, 1, 0.5],
#                               [0.5, 0, 1, 0, 0, 0, 1, 0, 0.5],
#                               [0.5, 1, 0, 1, 0.5, 0, 0, 0, 0]],
#                  evidence=['Contestant', 'Prize'], evidence_card=[3, 3])

cpd_c = TabularCPD('Contestant', 4, [[0.25], [0.25], [0.25], [0.25]])
cpd_p = TabularCPD('Prize', 4, [[0.25], [0.25], [0.25], [0.25]])
cpd_h = TabularCPD('Host', 4, [[0,   0,    0,    0,    0,    1/3, 0.5,  0.5,  0,    0.5,  1/3, 0.5,  0,    0.5,  0.5,  1/3],
                               [1/3, 0,    0.5,  0.5,  0,    0,   0,    0,    0.5,  0,    1/3, 0.5,  0.5,  0,    0.5,  1/3],
                               [1/3, 0.5,  0,    0.5,  0.5,  1/3, 0,    0.5,  0,    0,    0,   0,    0.5,  0.5,  0,    1/3],
                               [1/3, 0.5,  0.5,  0,    0.5,  1/3, 0.5,  0,    0.5,  0.5,  1/3, 0,    0,    0,    0,    0]],
                  evidence=['Contestant', 'Prize'], evidence_card=[4, 4])

# Associating the CPDs with the network structure.
model.add_cpds(cpd_c, cpd_p, cpd_h)


In [ ]:
# Infering the posterior probability
from pgmpy.inference import VariableElimination

infer = VariableElimination(model)
posterior = infer.query(variables=['Prize'], evidence={'Contestant': 0, 'Host': 2}, show_progress=False, joint=False)
print(posterior['Prize'])

+----------+--------------+
| Prize    |   phi(Prize) |
+==========+==============+
| Prize(0) |       0.2500 |
+----------+--------------+
| Prize(1) |       0.3750 |
+----------+--------------+
| Prize(2) |       0.0000 |
+----------+--------------+
| Prize(3) |       0.3750 |
+----------+--------------+


## Discussions
Given the contestant chooses door 0 at the beginning and the host opens door 2 which has a goat behind, the Bayesian network is implemented and the result is shown below.
####Monty hall problem with 3 doors
                    +----------+--------------+
                    | Prize    |   phi(Prize) |
                    +==========+==============+
                    | Prize(0) |       0.3333 |
                    +----------+--------------+
                    | Prize(1) |       0.6667 |
                    +----------+--------------+
                    | Prize(2) |       0.0000 |
                    +----------+--------------+
The model suggests that the contestant should switch to door 1, as it will double the winning probability. Sticking with the original choice (door 0) only has a 33.33% winning probability but switching to door 1 has a 66.67% winning probability.

####Monty hall problem with 4 doors
                   +----------+--------------+
                   | Prize    |   phi(Prize) |
                   +==========+==============+
                   | Prize(0) |       0.2500 |
                   +----------+--------------+
                   | Prize(1) |       0.3750 |
                   +----------+--------------+
                   | Prize(2) |       0.0000 |
                   +----------+--------------+
                   | Prize(3) |       0.3750 |
                   +----------+--------------+
The model suggests that the contestant should switch to door 1 or door 3. Sticking with the original choice (door 0) only has a 25% winning rate but switching to door 1 or door 3 has a 37.5% winning rate.
